In [1]:
!pip install pyshp

  Running setup.py bdist_wheel for pyshp: started
  Running setup.py bdist_wheel for pyshp: finished with status 'done'
  Stored in directory: C:\Users\Java\AppData\Local\pip\Cache\wheels\b8\a2\56\da058e1e1639061be3399baa32fb82362e2ae1afd8fe0179b0
Successfully built pyshp


https://gist.github.com/frankrowe/6071443

In [9]:
import shapefile
# read the shapefile
reader = shapefile.Reader("COUNTY_MOI_1060525.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
   atr = dict(zip(field_names, sr.record))
   geom = sr.shape.__geo_interface__
   buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 

# write the GeoJSON file
from json import dumps
geojson = open("pyshp-demo.json", "w")
geojson.write(dumps({"type": "FeatureCollection",\
"features": buffer}, indent=2) + "\n")
geojson.close()


TypeError: Object of type 'bytes' is not JSON serializable

https://gist.github.com/agalea91/c0e0d1897d1d98a0029ac0baa02b9fca

In [8]:
# -*- coding: utf-8 -*-
from json import dumps
import shapefile
import click

@click.command()
@click.option(
    '--f',
    help="Filename",
    default='COUNTY_MOI_1060525.shp'
)
def main(f):
    # read the shapefile
    reader = shapefile.Reader(f)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        record = sr.record
        # Make sure everything is utf-8 compatable
        record = [r.decode('utf-8', 'ignore') if isinstance(r, bytes) 
                  else r for r in record]
        atr = dict(zip(field_names, record))
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file
    with open('_'.join(f.split('.')[:-1]) + ".geo.json", "w") as geojson:
        geojson.write(dumps({"type": "FeatureCollection",\
                             "features": buffer}, indent=2) + "\n")

if __name__ == '__main__':
    main()

UnsupportedOperation: not writable

In [1]:
!pip install plotly

  Running setup.py bdist_wheel for plotly: started
  Running setup.py bdist_wheel for plotly: finished with status 'done'
  Stored in directory: C:\Users\Java\AppData\Local\pip\Cache\wheels\be\f1\88\edfe7e35fe42d192427ee868b952dd1af761367aceffb78e54
Successfully built plotly


In [2]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df.head()

df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

Aw, snap! We don't have an account for ''. Want to try again? You can authenticate with your email address or username. Sign in is not case sensitive.

Don't have an account? plot.ly

Questions? support@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.